In [34]:
from transformers import pipeline, set_seed
import re

In [35]:
def extract_after_first_string(text, first_string):
    pattern = re.escape(first_string) + r"(.+)"
    match = re.search(pattern, text, re.DOTALL)  # Add re.DOTALL flag to match newline characters
    if match:
        extracted_text = match.group(1)
        return extracted_text
    return None

def gpt2_generate(command):
    match = re.match(r"!gpt2_generate:(.+)", command)
    content = match.group(1)

    generator = pipeline('text-generation', model='gpt2')
    set_seed(42)
    gpt_output = generator(content, max_length=80, num_return_sequences=1)

    generated_text = gpt_output[0]['generated_text']  # Access the generated text from the output
    return generated_text
    # return extract_after_first_string(generated_text, content)




In [36]:
print(gpt2_generate("!gpt2_generate: make me a poem about a 1997 honda accord"))

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 make me a poem about a 1997 honda accordion accident. The whole series is all on the Internet at http://puu.sh/z6sKM. In another post about the incident, she states that she found a text message from one of her friends who is also an accordion player in a room. She says that the text wasn't about the accordion and that he


In [1]:
from crawler import crawl
from search import tfidf_search, wn_search, train
from generator import content_generator, gpt2_generate

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-06-01 15:06:25.004213: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-01 15:06:25.216854: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-01 15:06:25.218269: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 15:06:26.045959: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package wordnet to /home/jj/nltk_data..

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 embedding (Embedding)       (None, 9, 15)             75000     
                                                                 
 lstm (LSTM)                 (None, 15)                1860      
                                                                 
 dense (Dense)               (None, 5000)              80000     
                                                                 
 softmax (Softmax)           (None, 5000)              0         
                                                                 
Total params: 156,860
Trainable params: 156,860
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20


/home/jj/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
2023-06-01 15:09:32.658961: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [259,3]
	 [[{{node Placeholder/_0}}]]


259/259 [==============================] - 12s 46ms/step - loss: 2.3981 - accuracy: 0.6628
Epoch 2/20
259/259 [==============================] - 12s 46ms/step - loss: 1.3839 - accuracy: 0.7014
Epoch 3/20
259/259 [==============================] - 12s 46ms/step - loss: 1.1084 - accuracy: 0.7439
Epoch 4/20
259/259 [==============================] - 12s 46ms/step - loss: 1.0902 - accuracy: 0.7375
Epoch 5/20
259/259 [==============================] - 12s 45ms/step - loss: 1.0589 - accuracy: 0.7465
Epoch 6/20
259/259 [==============================] - 12s 46ms/step - loss: 1.1050 - accuracy: 0.7400
Epoch 7/20
259/259 [==============================] - 12s 46ms/step - loss: 0.9985 - accuracy: 0.7490
Epoch 8/20
259/259 [==============================] - 12s 46ms/step - loss: 0.9127 - accuracy: 0.7465
Epoch 9/20
259/259 [==============================] - 12s 45ms/step - loss: 0.8361 - accuracy: 0.7619
Epoch 10/20
259/259 [==============================] - 12s 45ms/step - loss: 0.7806 - accurac

In [16]:
command = "!wn_search 1997 honda accord" #!search 1997 honda accord

print(tfidf_search(command))

term:
1997 honda
words:
['1997', 'honda']
documento:
115
resultado:
{115: 0.04272502907096549}
documento:
116
resultado:
{115: 0.04272502907096549, 116: 0.038007843656181144}
documento:
117
resultado:
{115: 0.04272502907096549, 116: 0.038007843656181144, 117: 0.023636700230816318}
documento:
119
resultado:
{115: 0.04272502907096549, 116: 0.038007843656181144, 117: 0.023636700230816318, 119: 0.03097327936123763}
documento:
120
resultado:
{115: 0.04272502907096549, 116: 0.038007843656181144, 117: 0.023636700230816318, 119: 0.03097327936123763, 120: 0.05352453492666733}
documento:
124
resultado:
{115: 0.04272502907096549, 116: 0.038007843656181144, 117: 0.023636700230816318, 119: 0.03097327936123763, 120: 0.05352453492666733, 124: 0.004993671919551646}
documento:
125
resultado:
{115: 0.04272502907096549, 116: 0.038007843656181144, 117: 0.023636700230816318, 119: 0.03097327936123763, 120: 0.05352453492666733, 124: 0.004993671919551646, 125: 0.056552819230880534}
documento:
126
resultado:
{

In [11]:
command = "!gpt2_generate a poem about a 1997 honda accord"

print(gpt2_generate(command))

AttributeError: 'NoneType' object has no attribute 'group'

In [21]:
import re
command = "!search pato"

terms = re.findall('\w+', command.lower())

print(terms[0])

search


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
from tqdm import tqdm
import pandas as pd
from nltk.corpus import wordnet
import nltk
import random
import tensorflow as tf
from keras.models import load_model

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.layers import Input, Dense, Activation, TimeDistributed, Softmax, TextVectorization, Reshape, RepeatVector, Conv1D, Bidirectional, AveragePooling1D, UpSampling1D, Embedding, Concatenate, GlobalAveragePooling1D, LSTM, Multiply
from keras.models import Model
import tensorflow as tf
import keras

nltk.download('wordnet')
nltk.download('omw')
nltk.download('omw-1.4')


DATASET = 'crawler_data.csv'
df = pd.read_csv(DATASET)


vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['body'])
index = dict()

for w in tqdm(vectorizer.vocabulary_.keys()):
    index[w] = dict()
    for j in range(tfidf.shape[0]):
        if tfidf[j, vectorizer.vocabulary_[w]] > 0:
            index[w][j] = tfidf[j, vectorizer.vocabulary_[w]]



[nltk_data] Downloading package wordnet to /home/jj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /home/jj/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jj/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
100%|██████████| 60001/60001 [03:04<00:00, 325.55it/s]


In [26]:

def buscar(palavras, indice):
    assert type(palavras)==list
    resultado = dict()
    for p in palavras:
        if p in indice.keys():
            for documento in indice[p].keys():
                if documento not in resultado.keys():
                    resultado[documento] = indice[p][documento]
                else:
                    resultado[documento] += indice[p][documento]
                
    return resultado

#2 
def n_relevantes(result_busca, n):
    res = []
    for key in result_busca.keys():
        res.append( (result_busca[key], key)) 

    res = sorted(res, reverse= True)[0 : n]

    return res

#3
def query(q_str, n, index):

    words = re.findall('\w+', q_str)
    res = buscar(words, index)
    res_n = n_relevantes(res, n)
    return res_n


def train(index = index):
    for w in tqdm(vectorizer.vocabulary_.keys()):
        index[w] = dict()
        for j in range(tfidf.shape[0]):
            if tfidf[j, vectorizer.vocabulary_[w]] > 0:
                index[w][j] = tfidf[j, vectorizer.vocabulary_[w]]


def content_filter(content):

    bad_words = 'datasets/bad_words.csv'
    good_words = 'datasets/words_pos.csv'

    bad_words = pd.read_csv(bad_words)
    good_words = pd.read_csv(good_words)


    good_words = good_words.drop(columns=['pos_tag'])
    good_words['IsBad'] = 0
    bad_words['IsBad'] = 1

    good_words_sample = good_words.sample(1618, random_state=42)

    words = pd.concat([good_words_sample, bad_words])

    X = words["word"]
    y = words["IsBad"]

    X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2, random_state=42)

    classificador = Pipeline([
                        ('meu_vetorizador', CountVectorizer(stop_words='english')),
                        ('meu_classificador', LogisticRegression(penalty=None, solver='saga', max_iter=10000))
                        ])
    
    classificador.fit(X_train,y_train)
    y_pred = classificador.predict(X_test)
    acc = accuracy_score(y_pred,y_test)

    prob = classificador.predict_log_proba([content])
    probas = classificador.predict_proba([content])

    if prob[0][1] >= 1:
        return 1
    elif prob[0][1] <= -1:
        return -1
    

    m = np.max(probas)
    prob = 2 * (m -prob[0][1]) / (2 * m) - 1

    return prob #[0][1]


In [45]:

def tfidf_search(command):
    pattern = r"!search (.+)"
    pattern2 = r"th=(\d+(\.\d+)?)"

    term = ""
    threshold = None

    groups = re.match(pattern, command)
    
    if groups:
        term = groups.group(1)

        threshold_match = re.search(pattern2, command)

        if threshold_match:
            threshold = float(threshold_match.group(1))


    print("Term:", term)
    if threshold is not None:
        print("Threshold:", threshold)

    
    #aqui usamos tudo acima para pegar o documento com maior tf-idf, com indice invertido
    result = query(term, 1, index)
    print("result:")
    print(result)

    if result:
        # print(result[0][1])
        url = df.loc[result[0][1]].url
        content = df.loc[result[0][1]].body

        if threshold is not None:
            th = content_filter(content)
            if th < threshold:
                return "resultado abaixo do threshold especificado :("
            
        return (url, content)
    
    return "Nao Encontrado"


command = "!search carro" #!search 1997 honda accord

print(tfidf_search(command))

Term: carro
result:
[(0.06291479869093083, 57)]
('https://www.estadao.com.br/politica/', 'Política | Notícias do governo, STF e Congresso  - Estadão\n\n AssineNotícias & EditoriasOpiniãoÚltimasPolíticaBroadcast PolíticoColunasBlogsEconomiaAgronegóciosEstadão RIGovernançaIndicadoresMídia & MktNegóciosSua CarreiraE-InvestidorColunasBlogsPMEBrasilSão PauloColunasBlogsInternacionalThe New York Times Life/StyleColunasCulturaArtesAliásCinemaEstado da ArteLiteraturaModaMúsicaQuadrinhosSériesTeatro e DançaTelevisãoColunasBlogsEsportesCampeonatosBasqueteFutebolTênisUFCVelocidadeVôleiColunasBlogsProgramação de TVEstadão VerificaInfográficos e EspeciaisSaúdeCoronavírusAcervoAliásBlogsCiênciaColunasE+EducaçãoJornal do CarroLinkRecomendaMobilidadePaladarSustentabilidadeSoluções AmbientaisViagemTudo SobreVer & OuvirPodcastsTV EstadãoFotosRádio EldoradoDia a diaCruzadasHoróscopoLoteriasNewslettersQuadrinhosSudokuTempoParceirosBora.aíBrazil HealthCapituInstituto Vencer o CâncerJornal da USPMar Sem Fim

In [49]:

def wn_search(command):
    url = 'none'
    max_value = 0

    pattern = r"!wn_search (.+)"
    pattern2 = r"th=(\d+(\.\d+)?)"

    term = ""
    threshold = None
    content = ""

    groups = re.match(pattern, command)
    if groups:
        term = groups.group(1)

        threshold_match = re.search(pattern2, command)
        if threshold_match:
            threshold = float(threshold_match.group(1))

    print("Term:", term)
    if threshold is not None:
        print("Threshold:", threshold)

    # if threshold is not None:
    #     threshold = threshold.group(1)
    #     threshold = float(threshold)

    synsets = wordnet.synsets(term, lang='por')
    print([syn for syn in synsets])
    print([syn.name() for syn in synsets])
    print([syn.definition() for syn in synsets])

    #aqui usamos tudo acima para pegar o documento com maior tf-idf, com indice invertido
    result = query(term, 1, index)
    # print(result)

    if result:
        url = df.loc[result[0][1]].url
        max_value = result[0][0]
   
    for syn in synsets:

        definition = syn.definition()
        result = query(definition, 1, index)

        if result:
            value = result[0][0]

            if value > max_value:
                url = df.loc[result[0][1]].url
                content = df.loc[result[0][1]].body


    if url != 'none':
        if threshold is not None:

            th = content_filter(content)

            if th < threshold:
                return "resultado acima do threshold especificado :("
            
        return (url, content)
    
    return "Nao Encontrado"




command = "!wn_search carro th=0.001" #!search 1997 honda accord

print(wn_search(command))

Term: carro th=0.001
Threshold: 0.001
[]
[]
[]


UnboundLocalError: local variable 'content' referenced before assignment